In [77]:
import pandas as pd

In [ ]:
df = pd.read_csv('all.csv', header=0, encoding='utf-8')

In [ ]:
df.head()

In [ ]:
import json

df['tag_type'] = df['tags'].apply(lambda x: 'list' if '[' in x else 'str')
df['tag_type'].value_counts()

In [ ]:
df[df['tag_type'] == 'list'].head()

In [ ]:
df.loc[(df.tag_type == 'list'), 'tags'] = df.loc[(df.tag_type == 'list'), 'tags'].apply(lambda x: ";".join(json.loads(x.replace('\'', '"'))))

In [ ]:
df.loc[(df.tag_type == 'list'), 'tags']

In [ ]:
df = df.drop(columns=['tag_type'])
df.to_csv('all_v2.csv', index=False)

In [ ]:
df = pd.read_csv('all_v2.csv', header=0, encoding='utf-8')

In [ ]:
df.head()

In [ ]:
freq = pd.Series((" ".join(df.loc[df.tags.notnull(), 'tags'].str.replace(';', ' '))).split()).value_counts()
freq.to_csv('freq.csv')

In [ ]:
df.tags.apply(lambda x: 'str' if isinstance(x, str) else 'non_str').value_counts()

In [ ]:
df.tags.isna()

In [ ]:
freq = pd.read_csv('freq.csv', header=0)

In [ ]:
freq.loc[:50,]

In [ ]:
with open('freq.json', 'w', encoding='utf-8') as f:
    freq.to_json(f, orient="index", force_ascii=False, indent=2)

In [ ]:
def get_freq(df):
    return pd.Series((" ".join(df.loc[df.tags.notnull(), 'tags'].str.replace(';', ' '))).split()).value_counts()[:50]

grps = df.groupby(['KADAA_ID', 'KADAA_AR', 'KADAA_EN'])
res = {}
for name, group_df in grps:
    _id, ar, en = name
    res[int(_id)] = {
        'KADAA_AR': ar,
        'KADAA_EN': en,
        'trending': list(get_freq(group_df).index)
    }

In [ ]:
import json
with open('kaddaas_trending.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(res, indent=2, ensure_ascii=False))

In [ ]:
topics = df[['text', 'date', 'KADAA_ID', 'KADAA_AR', 'KADAA_EN', 'MOHAFAZA_ID', 'MOHAFAZA_AR', 'MOHAFAZA_EN']].copy()
topics

In [ ]:
def n_grams(s, n_range):
    ngrams = []
    words = s.split()
    length = len(words)
    for n in n_range:
        for i in range(length):
            if i + n <= length:
                ngrams.append(" ".join(words[i: i + n]))
    return ngrams


In [ ]:
s = topics['text'].apply(lambda x: n_grams(x, [2, 3])).apply(pd.Series, 1).stack()
s.index = s.index.droplevel(-1) # to line up with df's index
s.name = 'text' # needs a name to join
del topics['text']
topics = topics.join(s).reset_index()
topics

In [ ]:
topics = topics.drop(columns=['index'])
topics.columns

In [ ]:
topics

In [ ]:
topics.to_csv('topics.csv', index=False)

In [ ]:
topics.text.value_counts().reset_index()

In [79]:
topics = pd.read_csv('topics.csv', header=0)
# topics[['year', 'month', 'day']] = topics['date'].str.split('-', expand=True)
topics

,date,KADAA_ID,KADAA_AR,KADAA_EN,MOHAFAZA_ID,MOHAFAZA_AR,MOHAFAZA_EN,text,year,month,day
0,2020-07-12,83000,صور,Sour,8,الجنوب,South,#كورونا يتلف,2020,7,12
1,2020-07-12,83000,صور,Sour,8,الجنوب,South,يتلف رئة,2020,7,12
2,2020-07-12,83000,صور,Sour,8,الجنوب,South,"رئة ""أبو",2020,7,12
3,2020-07-12,83000,صور,Sour,8,الجنوب,South,"""أبو عزرائيل""",2020,7,12
4,2020-07-12,83000,صور,Sour,8,الجنوب,South,"عزرائيل"" (صورة)",2020,7,12
...,...,...,...,...,...,...,...,...,...,...,...
2013306,2020-01-01,41000,زحلة,Zahle,4,البقاع,Bekaa,upcoming transformation of,2020,1,1
2013307,2020-01-01,41000,زحلة,Zahle,4,البقاع,Bekaa,transformation of the,2020,1,1
2013308,2020-01-01,41000,زحلة,Zahle,4,البقاع,Bekaa,of the 20s!,2020,1,1
2013309,2020-01-01,41000,زحلة,Zahle,4,البقاع,Bekaa,the 20s! https://twitter.com/SeanShirvaniMD/st...,2020,1,1


In [ ]:
topics.to_csv('topics.csv', index=False)

In [93]:
import json
res = {}
grp = topics.groupby(['year', 'month', 'KADAA_ID', 'KADAA_AR', 'KADAA_EN'])
for name, group in grp:
    df = group.reset_index().drop(columns=['index'])
    y,  m, _id, ar, en = name
    key = str(y) + '_' + str(m) + '_' + str(_id) + '_' + str(ar) + '_' + str(en)
    res[key] = list(df.text.value_counts()[:50].index)

with open('kadaas_topics.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(res, indent=2, ensure_ascii=False))